In [14]:
from pantheon import pantheon
import asyncio
import nest_asyncio
nest_asyncio.apply()

server = "euw1"
api_key = 'RGAPI-2094315c-31a1-4d3d-953f-e4441756364a'

def requestsLog(url, status, headers):
    print(url)
    print(status)
    print(headers)

panth = pantheon.Pantheon(server, api_key, errorHandling=True, requestsLoggingFunction=requestsLog, debug=True)

async def getSummonerId(name):
    try:
        data = await panth.getSummonerByName(name)
        return (data['id'],data['accountId'])
    except Exception as e:
        print(e)


async def getRecentMatchlist(accountId):
    try:
        data = await panth.getMatchlist(accountId, params={"endIndex":10})
        return data
    except Exception as e:
        print(e)

async def getRecentMatches(accountId):
    try:
        matchlist = await getRecentMatchlist(accountId)
        tasks = [panth.getMatch(match['gameId']) for match in matchlist['matches']]
        return await asyncio.gather(*tasks)
    except Exception as e:
        print(e)



In [15]:
name = "KC Isnow"

loop = asyncio.get_event_loop()  


In [16]:
(summonerId, accountId) = loop.run_until_complete(getSummonerId(name))

https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/KC Isnow
200
<CIMultiDictProxy('Content-Encoding': 'gzip', 'Content-Type': 'application/json;charset=utf-8', 'Date': 'Sat, 05 Jun 2021 11:30:44 GMT', 'Vary': 'Accept-Encoding', 'X-App-Rate-Limit': '20:1,100:120', 'X-App-Rate-Limit-Count': '1:1,24:120', 'X-Method-Rate-Limit': '2000:60', 'X-Method-Rate-Limit-Count': '2:60', 'X-Riot-Edge-Trace-Id': 'bfc882e3-20fa-4f57-9c5d-55cc413bacd0', 'Content-Length': '286', 'Connection': 'keep-alive')>


In [17]:
print("summonerId = ", summonerId)
print("accountId = ", accountId)


summonerId =  Z2bfMio01qPjPdWaMX_2Xr89PT2IDmi4iuHFbS2eazTGQl5Tdbe4aFLWcA
accountId =  1Q69y9VDie8nz__-dBgm_IlQ0McrVp3rrFI8U1_Lrdbmzi0rHNOBTmxK


In [18]:
print(loop.run_until_complete(getRecentMatches(accountId)))

https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/1Q69y9VDie8nz__-dBgm_IlQ0McrVp3rrFI8U1_Lrdbmzi0rHNOBTmxK?endIndex=10
200
<CIMultiDictProxy('Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'Content-Type,Authorization,Region,Cookie', 'Access-Control-Allow-Methods': 'GET,POST,OPTIONS', 'CF-Cache-Status': 'DYNAMIC', 'CF-RAY': '65a912197e6534c0-DUB', 'cf-request-id': '0a7d89a3ed000034c0409c0000000001', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json;charset=utf-8', 'Date': 'Sat, 05 Jun 2021 11:30:45 GMT', 'Date': 'Sat, 05 Jun 2021 11:30:45 GMT', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'X-App-Rate-Limit': '20:1,100:120', 'X-App-Rate-Limit-Count': '1:1,25:120', 'X-Method-Rate-Limit': '1000:10', 'X-Method-Rate-Limit-Count': '3:10', 'X-Riot-Edge-Trace-Id': '181f9497-dee5-4f7b-9a9d-3f3c52581fcd', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive')>
https://euw1.api.r

In [19]:
async def getMatchAndTimeline(gameId):
    match, timeline = await asyncio.gather(*[
        panth.getMatch(gameId),
        panth.getTimeline(gameId)
    ])
    match['timeline'] = timeline
    return match

In [20]:
gamesDataRow = loop.run_until_complete(getRecentMatches(accountId))

https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/1Q69y9VDie8nz__-dBgm_IlQ0McrVp3rrFI8U1_Lrdbmzi0rHNOBTmxK?endIndex=10
200
<CIMultiDictProxy('Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'Content-Type,Authorization,Region,Cookie', 'Access-Control-Allow-Methods': 'GET,POST,OPTIONS', 'CF-Cache-Status': 'DYNAMIC', 'CF-RAY': '65a91229fdb234d8-DUB', 'cf-request-id': '0a7d89ae3e000034d865930000000001', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json;charset=utf-8', 'Date': 'Sat, 05 Jun 2021 11:30:47 GMT', 'Date': 'Sat, 05 Jun 2021 11:30:48 GMT', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'X-App-Rate-Limit': '20:1,100:120', 'X-App-Rate-Limit-Count': '1:1,36:120', 'X-Method-Rate-Limit': '1000:10', 'X-Method-Rate-Limit-Count': '1:10', 'X-Riot-Edge-Trace-Id': '122d70e0-4507-49f6-9f7c-9fcca351dc71', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive')>
https://euw1.api.r

In [21]:
def getParticipantIdFromAccountId(game,accountId):
    for participant in game['participantIdentities']:
        if participant['player']['currentAccountId'] == accountId:
            return participant['participantId']

participantId =getParticipantIdFromAccountId(gamesDataRow[0],accountId)
participantId

3

In [22]:
def getTeamAndChampion(game, participantId):
    for participant in game['participants']:
        if participant['participantId'] == participantId:
            return participant['teamId'], participant['championId'], participant['stats']['win']

teamId, championId, win = getTeamAndChampion(gamesDataRow[0], participantId)
print(teamId, championId, win)

100 28 True


In [23]:
def getTeamComposition(game):
    #creer une liste pour chaque equipe
    compositions = {100:[], 200:[]}
    for participant in game['participants']:
        compositions[participant['teamId']].append(participant['championId'])
    return compositions
compositions = getTeamComposition(gamesDataRow[0])
compositions

{100: [12, 157, 28, 223, 18], 200: [80, 40, 81, 427, 150]}

In [24]:
def getSkillOrder(games, participantId):
    skillOrder = []
    for frame in game['timeline']['frames']:
        for event in frame['events']:
            if event['type'] == 'SKILL_LEVEL_UP' and event['participantId']== participantId:
                skillOrder.append(str(event['skillslot']))
    return " > ".join(skillOrder)

#skillOrder = getSkillOrder(gamesDataRow[0], participantId)
#skillOrder


In [25]:
playerChampion = championIdToName[championId]
teamComposition = [championIdToName[chId] for chId in compositions[teamId]]
enemyTeamComposition = [championIdToName[chId] for chId in compositions[100 if teamId==200 else 200]]
print("Ally team: ","/".join(teamComposition))
print("Enemy team: ","/".join(enemyTeamComposition))

NameError: name 'championIdToName' is not defined

In [26]:
response = requests.get('http://ddragon.leagueoflegends.com/cdn/11.10.1/data/en_US/item.json')
ItemRawData = json.loads(response.text)

itemIdToName= {}
for key,item in ItemRawData['data'].items():
    itemIdToName[int(key)] = item['name']

def getItemList(game, participantId):
    for participant in game['participants']:
        itemsList = []
        for i in range(0, 7):
            if not participant['stats']['item' + str(i)] == 0:
                itemsList.append(itemIdToName[participant['stats']['item' + str(i)]])
        return " / ".join(itemsList)
getItemList(gamesDataRow[4], participantId)

NameError: name 'requests' is not defined

In [27]:
def getPatch(game):
    return ".".join(game['gameVersion'].split(".")[:2])

getPatch(gamesDataRow[0])

'11.11'

In [28]:
import datetime

def getDate(game):
    return datetime.datetime.fromtimestamp(int(game["gameCreation"]/1000)).strftime('%D')

getDate(gamesDataRow[0])

'06/05/21'

In [29]:
gameInformationList = []

for game in gamesDataRow:
    #récuperer info
    participantId = getParticipantIdFromAccountId(game, accountId)
    teamId, championId, win = getTeamAndChampion(game, participantId)
    compositions = getTeamComposition(game)

    #info dans la liste 
    gameInformation = {}
    
    gameInformation['champion'] = championIdToName[championId]
    gameInformation['patch'] = getPatch(game)
    gameInformation['date'] = getDate(game)
    gameInformation['lvlups'] = getSkillOrder(game, participantId)
    gameInformation['summoners 1'] = gamesDataRow[0]['participants'][2]['spell1Id']
    gameInformation['summoners 2'] = gamesDataRow[0]['participants'][2]['spell2Id']
    gameInformation['teamComposition'] = "/".join([championIdToName[chId] for chId in compositions[teamId]])
    gameInformation['enemyTeamComposition'] ="/".join([championIdToName[chId] for chId in compositions[100 if teamId == 200 else 200]])
    gameInformation['win'] = win
    #ajouter la ligne a la liste
    gameInformationList.append(gameInformation)

gameInformationList

NameError: name 'championIdToName' is not defined

In [ ]:
import pandas as pd
df = pd.DataFrame(gameInformationList)
df.head(51)

In [164]:
gamesDataRow[0]['participants']

[{'participantId': 1,
  'teamId': 100,
  'championId': 23,
  'spell1Id': 6,
  'spell2Id': 4,
  'stats': {'participantId': 1,
   'win': True,
   'item0': 1055,
   'item1': 3006,
   'item2': 6672,
   'item3': 3153,
   'item4': 1036,
   'item5': 0,
   'item6': 3513,
   'kills': 4,
   'deaths': 3,
   'assists': 1,
   'largestKillingSpree': 4,
   'largestMultiKill': 2,
   'killingSprees': 1,
   'longestTimeSpentLiving': 377,
   'doubleKills': 1,
   'tripleKills': 0,
   'quadraKills': 0,
   'pentaKills': 0,
   'unrealKills': 0,
   'totalDamageDealt': 124382,
   'magicDamageDealt': 0,
   'physicalDamageDealt': 110515,
   'trueDamageDealt': 13866,
   'largestCriticalStrike': 1567,
   'totalDamageDealtToChampions': 10875,
   'magicDamageDealtToChampions': 0,
   'physicalDamageDealtToChampions': 9649,
   'trueDamageDealtToChampions': 1226,
   'totalHeal': 5389,
   'totalUnitsHealed': 1,
   'damageSelfMitigated': 10754,
   'damageDealtToObjectives': 23620,
   'damageDealtToTurrets': 3908,
   'vis